In [210]:
#imports

import pandas as pd
import numpy as np
import matplotlib
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math


In [211]:
#create dataset of points
#f=open('/Users/benso/Desktop/Projects/MachineLearning/Assignment3/hw3_data2.txt',"r")
f=open('/Users/lvbenson/Research_Projects/MachineLearning/Assignment3/hw3_data2.txt',"r")

lines = f.readlines()
result=[]
for x in lines:
    result.append(x.split(',')[0])
f.close()

target = []
for sign in result:
    if sign == '+':
        target.append(1)
    else:
        target.append(-1)

#f=open('/Users/benso/Desktop/Projects/MachineLearning/Assignment3/hw3_data2.txt',"r")
f=open('/Users/lvbenson/Research_Projects/MachineLearning/Assignment3/hw3_data2.txt',"r")
lines1=f.readlines()
result2=[]
for x in lines1:
    result2.append(x.split(',')[1])
f.close()

x_coords = []
for coord in result2:
    x_coords.append(coord)

#f=open('/Users/benso/Desktop/Projects/MachineLearning/Assignment3/hw3_data2.txt',"r")
f=open('/Users/lvbenson/Research_Projects/MachineLearning/Assignment3/hw3_data2.txt',"r")
lines2=f.readlines()
result3=[]
for x in lines2:
    x_ = x.split(',')[2].rstrip('\n')
    result3.append(x_)
y_coords = []
for coord in result3:
    y_coords.append(coord)
data = []
for x,y in zip(x_coords,y_coords):
    data.append([x,y])
f.close()

X = np.array(data,dtype=float)
Y = np.array(target,dtype=float)

In [212]:
#create test, train vectors
x_train,x_test,y_train,y_test = train_test_split(X,Y)

In [213]:
def rbf_kernel(X,gamma):
    K = np.zeros((X.shape[0],X.shape[0]))
    for i in range(X.shape[0]): 
        for j in range(X.shape[0]):
            K[i,j] = np.exp(-gamma*np.linalg.norm(X[i]-X[j])**2) # rbf kernel alg, kinda like euclid distance. All this function does it calculate the kernel, to be used later.
    return K

In reference to this document: http://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf, and this document: https://towardsdatascience.com/optimization-loss-function-under-the-hood-part-iii-5dff33fa015d, the following is the implementation of a function to determine the cost for each class. This function uses the weight vector calculated in the SVM function, and then goes through the dataset to determine the cost based on the corresponding element classified in the target vector (Y). 

As I learned from the towardsdatascience document, the following is the SVM cost function:

$Cost(h_{\theta},(x),y)=max(0,1-\theta^{T}x)$ if y = 0 
and
$Cost(h_{\theta},(x),y)=max(0,1+\theta^{T}x)$ if y = 1

Where y is our classification. In my case, the two possible classifications are -1 and 1.

So, the final loss function for the non-linear SVM is as follows (and is calculated in the loss calculation function). Note that for this implementation, I chose to use a regularized C term of just 1, but this can be manipulated. This could be used to see how, over time, the loss improves as the line wiggles less and less. If I have time, I'll try to visualize this loss function.

(also retrieved from the towardsdatascience document):

$J(\theta)=C[\sum_{i=1}^{m}y^{i}Cost_{1}(\theta^{T}(f^{(i)})+(1-y^{(i)})Cost_{0}(\theta^{T}(f^{(i)})]$











In [214]:
def Loss_Calculation(w,K,X,Y,reg_term=1,C=1): #K is 2700 x 2700 matrix (for training data), x is 2700x2, w is 2700x1 (training data)
    #Hinge loss determinism
    L_list = []
    size_y = range(len(y))
    for i in size_y:
        if -Y[i]*classification(i,w,Y,K)<1:
            L_list.append(i)

    for i in size_y:
        for j in L_list:
            sub_grad_help = sum([Y[j]*K[j][i]])
        sub_grad_list = [(-Y[i]*sub_grad_help)]
        w_change = np.array(sub_grad_list)

    return w_change

In [215]:
#this is our overall classification function that we'll use to check to see if the point is classified as positive or negative.
def classification(index_x,w,Y,K):
    size_y = range(len(Y))
    class_func = 0
    for i in size_y:
        class_func += Y[i]*K[index_x][i]*w[i]
    return class_func

In [216]:
def SVM(X,Y,K,epochs=25,learn_rate=1,reg_term=1,C=1,Gamma=1):
    #initialize random weights
    w = np.zeros(X.shape[0])
    for epoch in range(epochs):
        w_change = Loss_Calculation(w,K,X,Y,reg_term,C)
        w = np.subtract(w,w_change) #account for the wiggle in the w vector
    
    return w

In [217]:
def pipeline(X,Y,epochs=25,learn_rate=1,reg_term=1,C=1,Gamma=1):
    K = rbf_kernel(X,Gamma)
    w_vector = SVM(X,Y,K,epochs,learn_rate)
    #need to classify everything in X 
    #things to classify:
    classifications = [classification(i,w_vector,Y,K) for i in range(len(X))]
    """
    size = range(len(X))

    classify_list = []
    for index_x in size:
        classify_list.append(classification(index_x,w_vector,Y,K,))
    """
    return classifications,w_vector

classes,w = pipeline(x_train,y_train)
correct_classify = []
incorrect_classify = []

In [218]:
#check out the training data accuracy

for i,y in zip(classes,y_train):
    if i > 0 and y == 1:
        correct_classify.append(1)
    elif i < 0 and y == -1:
        correct_classify.append(1)
    else:
        incorrect_classify.append(1)
total_correct = (len(correct_classify)) / (len(correct_classify) + len(incorrect_classify))
print(total_correct,' :accuracy of training data')


0.9533333333333334  :accuracy of training data


In [219]:
#check out the test data accuracy. only calculate kernel and classes, use the w that is already calculated
K = rbf_kernel(x_test,1)
classifications_test = [classification(i,w,y_test,K) for i in range(len(x_test))]

correct_classify_test = []
incorrect_classify_test = []

#check out the training data accuracy

for i,y in zip(classifications_test,y_test):
    if i > 0 and y == 1:
        correct_classify_test.append(1)
    elif i < 0 and y == -1:
        correct_classify_test.append(1)
    else:
        incorrect_classify_test.append(1)
total_correct_test = (len(correct_classify_test)) / (len(correct_classify_test) + len(incorrect_classify_test))
print(total_correct_test,' :accuracy of test data')

0.9711111111111111  :accuracy of test data
